# Generates Mobility file for inference

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')
    
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import networkx as nx
import copy
import scipy as sp
import math
import seaborn
import pickle
import warnings
import os
import random

from lib.mobilitysim import MobilitySimulator
from lib.town_data import generate_population, generate_sites, assign_work_sites, compute_distances
from lib.town_maps import MapIllustrator

### Settings for synthetic mobility data generation

Import __one__ `town_settings` file. The following variables will be imported by the `import *` command
* `town_name`
* `population_path`
* `sites_path`
* `bbox`
* `population_per_age_group`
* `region_population`
* `town_population`
* `daily_tests_unscaled`
* `household_info`

In [ ]:
# from lib.settings.town_settings_kaiserslautern import *
# from lib.settings.town_settings_ruedesheim import *
# from lib.settings.town_settings_tirschenreuth import *
# from lib.settings.town_settings_tubingen import *
from lib.settings.town_settings_sanfrancisco import *

# from lib.settings.town_settings_lausanne import *
# from lib.settings.town_settings_locarno import *
# from lib.settings.town_settings_lucerne import *
# from lib.settings.town_settings_jura import *

In [ ]:
# Downsampling factor of population and sites
downsample = 100

# Country for different age groups
country = 'US' # 'GER', 'CH'

# Set the population generation mode.
# 3 options available: custom | random | heuristic
population_by = 'custom'

### Nothing should be changed below

---

#### Town details

In [ ]:
# Downsample population 
population_per_age_group = np.round(
    population_per_age_group * (town_population / (downsample * region_population))).astype('int').tolist()

print(f'Population per age group: {population_per_age_group}')

In [ ]:

model_essential=True
if model_essential == True:
    '''Hack: Must be changed if want to model essential workers not in SF'''
    from lib.settings.town_settings_sanfrancisco import _essential_distribution, _worker_mobility

    # proportion of total population that are essential workers
    essential_to_total_ratio = 0.2
    
    # which worker types to include (0:education, 1:office, 2:retail stores, 3:social, 4:supermarket)
    incl_worker_types = [0,1,2,3,4]
        
    essential_distribution = _essential_distribution()
    num_essential_workers = np.floor(sum(population_per_age_group)*essential_to_total_ratio).astype('int').tolist()
    num_essential_per_age_group = np.floor(num_essential_workers * essential_distribution).astype('int')
    essential_prop_per_age_group = np.divide((num_essential_per_age_group),(population_per_age_group))   

    strFormat = len(essential_prop_per_age_group) * '{:.2%} '
    print(f'Proportion of age groups that are essential workers:\n {strFormat.format(*essential_prop_per_age_group)}')
    print(f'Essential workers per age group:\n {num_essential_per_age_group}')
    print(f'Proportion of essential workers to total: {float((num_essential_per_age_group).sum())/sum(population_per_age_group):.3%}')
else:
    essential_prop_per_age_group = None
    worker_types = None

#### Extracted site data

* `site_loc`: list of site coordinates
* `site_type`: list of site category
* `site_dict`: helper dictionary with real name (string) of each site category (int)
* `density_site_loc`: list of site coordinates of specific type to be based on to generate population density

To generate sites of arbitrary sites for a given city, the following function sends queries to OpenStreetMap. In order to use it for additional types of sites, you need to specify queries in the Overpass API format. For more information, check the existing queries in **/lib/data/queries/**, https://wiki.openstreetmap.org/wiki/Overpass_API and http://overpass-turbo.eu/.

We separatelly use a query returning all buildings in a town to heuristically generate population density in the next steps if no real population density data is provided. An extra query is required for this purpose and it should be given as a **site_based_density_file** argument.

In [ ]:
# This block sends queries to OpenStreetMap
# Make sure you have a working internet connection
# If an error occurs during execution, try executing again 
# If the call times out or doesn't finish, try restarting your internet connection by e.g. restarting your computer
site_files=[]
for root,dirs,files in os.walk(sites_path):
    for f in files:
        if f.endswith(".txt") and f != 'buildings.txt':
            site_files.append(sites_path+f)

site_loc, site_type, site_dict, density_site_loc = generate_sites(bbox=bbox, query_files=site_files,sites_path=sites_path,
                                site_based_density_file=sites_path+'buildings.txt')

In [ ]:
# before downsampling
print(site_dict)
print('Number of sites: ',np.sum(np.array(site_type)==0),
                          np.sum(np.array(site_type)==1),
                          np.sum(np.array(site_type)==2),
                          np.sum(np.array(site_type)==3),
                          np.sum(np.array(site_type)==4))

#### Site visualization

In [ ]:
# ill = MapIllustrator()
# sitemap = ill.sites_map(bbox=bbox, site_loc=site_loc, site_type=site_type, site_dict = site_dict, map_name=f'{town_name}_site_distribution')
# sitemap

#### Generate home location based on various options

* `home_loc`: list of home coordinates
* `people_age`: list of age category 
* `home_tile`: list of map tile to which each home belongs
* `tile_loc`: list tile center coordinates

The following three options generate a population distribution across a geographical area consisting of tiles (square boxes) of specific resolution. More information about tile sizes can be found in https://wiki.openstreetmap.org/wiki/Zoom_levels. 

In [ ]:
if region_population == town_population:
    tile_level = 15
else:
    tile_level = 16

if population_by == 'custom':
    # generate population across tiles based on density input
    print('Tile level: ', tile_level)
    home_loc, people_age, home_tile, tile_loc, people_household, worker_types = generate_population(
        density_file=population_path, bbox=bbox,
        population_per_age_group=population_per_age_group, 
        household_info=household_info, tile_level=tile_level, seed=42,
        essential_prop_per_age_group=essential_prop_per_age_group,
        incl_worker_types=incl_worker_types)
    
elif population_by == 'random':
    # generate population across tiles uniformly at random
    home_loc, people_age, home_tile, tile_loc, people_household, worker_types = generate_population(
        bbox=bbox, population_per_age_group=population_per_age_group,
        tile_level=16, seed=42,
        essential_prop_per_age_group=essential_prop_per_age_group,
        incl_worker_types=incl_worker_types)

elif population_by == 'heuristic':
    # generate population across tiles proportional to buildings per tile
    home_loc, people_age, home_tile, tile_loc, people_household, worker_types = generate_population(
        bbox=bbox, density_site_loc=density_site_loc,
        population_per_age_group=population_per_age_group, tile_level=16, seed=42,
        essential_prop_per_age_group=essential_prop_per_age_group,
        incl_worker_types=incl_worker_types)

In [ ]:
print(f'Num essential workers: {(np.array(worker_types)!=-1).sum()}/{len(worker_types)}')

In [ ]:
unique_household, counts_household = np.unique(people_household, return_counts=True)
plt.hist(counts_household,bins=range(1,9),align='left',rwidth=0.5)
plt.xlabel('Household Size')
plt.ylabel('Number of Households')

In [ ]:
household_loc = []
household_loc_dict = {}
for i,ind in enumerate(people_household):
    household_loc_dict[ind] = home_loc[i]
for i in range(len(household_loc_dict)):
    household_loc.append(household_loc_dict[i])

#### Home visualization

In [ ]:
# homemap = ill.population_map(bbox=bbox, home_loc=home_loc, map_name=f'{town_name}_population_distribution')
# homemap # zoom in to see details

# Social Graph

In [ ]:
random.seed(42)
edges_out = 5
num_colleages = 10 # used in mobilitysim for adding colleages to social graph
num_people = len(people_age)
friendships = [random.sample(range(num_people), 2) for i in range(num_people * edges_out)]
social_graph = nx.Graph()
social_graph.add_nodes_from(range(num_people))
social_graph.add_edges_from(friendships)
num_friends = [social_graph.degree[i] for i in range(num_people)]
print('Number of edges: ', social_graph.number_of_edges())
print('Max friends: ', max(num_friends))
print('Min friends: ', min(num_friends))
plt.hist(num_friends,bins=20)
print(np.mean(num_friends),np.std(num_friends))

# Home Gathering

In [ ]:
refuse_gathering_rate = 0.3
gather_max_size = 10

Downsample sites as given by settings

In [ ]:
site_downsample = downsample

if site_downsample > 1:
#     np.random.seed(42)
#     # downsample sites like populatoin
#     idx = np.random.choice(len(site_loc), size=int(len(site_loc) / site_downsample), 
#                            replace=False, p=np.ones(len(site_loc)) / len(site_loc))

#     site_loc, site_type = np.array(site_loc)[idx].tolist(), np.array(site_type)[idx].tolist()

    # Zihan: new downsampling method so that each type is downsampled by the same value
    site_loc_downsampled = []
    site_type_downsampled = []
    for i in range(len(site_dict)):
        curr_type_all = np.zeros(len(site_type))
        curr_type_all[np.array(site_type)==i] = 1
        idx = np.random.choice(len(site_type), size=int(np.sum(np.array(site_type)==i) / site_downsample), 
                               replace=False, p=curr_type_all / np.sum(np.array(site_type)==i))
        site_loc_downsampled = site_loc_downsampled + np.array(site_loc)[idx].tolist()
        site_type_downsampled = site_type_downsampled + np.array(site_type)[idx].tolist()
    site_loc = site_loc_downsampled
    site_type = site_type_downsampled

# Append homes to sites

In [ ]:
# No downsampling for home loc
people_house_site = [ind + len(site_type) for ind in people_household] # site index of people's household
site_loc += household_loc
site_type += [len(site_dict)]*len(household_loc)
site_dict[len(site_dict)] = 'home'

In [ ]:
print(f'Number of sites: ', len(site_loc))
print(f'Site types:      ', site_dict)
print('Number of sites: ',np.sum(np.array(site_type)==0),
                          np.sum(np.array(site_type)==1),
                          np.sum(np.array(site_type)==2),
                          np.sum(np.array(site_type)==3),
                          np.sum(np.array(site_type)==4),
                          np.sum(np.array(site_type)==5))

#### Assign essential work sites

In [ ]:
worker_work_sites = assign_work_sites(worker_types,site_type)

Compute pairwise distances between all tile centers and all sites

In [ ]:
tile_site_dist = compute_distances(site_loc, tile_loc)

#### Specify synthetic mobility patterns

Here we specify the patterns of mobility used for generating the synthetic traces based on the above home and site locations. Note that this is a general framework and can by arbitrarilty extended to any desired site numbers or types. See below for an example used in the first version of our paper.

In [ ]:
# e.g. line 0 corresponds to age 0-4 in Germany
# a lot of eduction (kindergarden), some social, no public transport, no office, no supermarket
# the age groups are chosen to match the age groups used in case data by national authorities
# GERMANY
if country == 'GER':
    mob_rate_per_age_per_type = [
        [5, 1, 0, 0, 0], # 0-4
        [5, 2, 3, 0, 0], # 5-14
        [2, 2, 3, 3, 1], # 15-34
        [0, 2, 1, 5, 1], # 35-59
        [0, 3, 2, 0, 1], # 60-79
        [0, 2, 1, 0, 1]]  # 80+
    dur_mean_per_type = [2, 1.5, 0.2, 2, 0.5]
    variety_per_type = [1, 10, 5, 1, 2]

# SWITZERLAND
elif country == 'CH':
    mob_rate_per_age_per_type = [
       [5, 1, 0, 0, 0], # 0-9
       [5, 2, 3, 0, 0], # 10-19
       [2, 2, 3, 3, 1], # 20-29
       [2, 2, 3, 3, 1], # 30-39
       [0, 2, 1, 5, 1], # 40-49
       [0, 2, 1, 5, 1], # 50-59
       [0, 3, 2, 0, 1], # 60-69
       [0, 3, 2, 0, 1], # 70-79
       [0, 2, 1, 0, 1]] # 80+
    dur_mean_per_type = [2, 1.5, 0.2, 2, 0.5]
    variety_per_type = [1, 10, 5, 1, 2]
    
elif country == 'US':
    #  {0: 'education', 1: 'office', 2: 'retail', 3: 'social', 4: 'supermarket'}
#     mob_rate_per_age_per_type = [
#          [5,    0,    0, 0,   0,   0.5],    # 0-5
#          [5,    0,    0, 0,   0,   0.5],    # 5-14
#          [5,    0, 1.55, 3.6, 0.22,0.5], # 15-19
#          [1.48, 3.52, 1.44, 3.6, 0.21,0.5], # 20-24
#          [0,    5, 1.87,    3.6, 0.27,0.5], # 25-44
#          [0,    5, 2.46,    3.6, 0.36,0.5], # 45-59
#          [0,    0, 2.40,    3.6, 0.35,0.5], # 60-79
#          [0,    0, 2.43,    3.6, 0.35,0.5]] # 80+
    mob_rate_per_age_per_type = [ # calculated with updated Safegraph data in the week 2020-02-24 to 2020-03-01
         [5,    0,    0, 0,   0,   0],    # 0-5
         [5,    0,    0, 0,   0,   0.5],    # 5-14
         [5,    0, 1.16, 2.30, 0.22,0.5], # 15-19
         [1.48, 3.52, 1.16, 2.30, 0.20,0.5], # 20-24
         [0,    5, 1.16,    2.30, 0.26,0.5], # 25-44
         [0,    5, 1.16,    2.30, 0.35,0.5], # 45-59
         [0,    0, 1.16,    2.30, 0.34,0.5], # 60-79
         [0,    0, 1.16,    2.30, 0.34,0.5]] # 80+

#     dur_mean_per_type = [5.0, 5.0, 0.55, 0.64, 0.4, 3.0]
    dur_mean_per_type = [5.0, 5.0, 0.70, 0.83, 0.55, 3.0] # calculated with updated Safegraph data in the week 2020-02-24 to 2020-03-01
    variety_per_type = [1, 1, 10, 10, 2, 1] # variety_per_type for home sites does not matter
    
    if model_essential==True:
        wtype='-'.join([str(w) for w in incl_worker_types])
#         essential_mob_rate_per_type, essential_dur_mean_per_type, essential_variety_per_type = mob_rate_per_age_per_type[0], dur_mean_per_type, variety_per_type
#         essential_mob_rate_per_type, essential_dur_mean_per_type, essential_variety_per_type = _essential_mobility(wtype)
        worker_mob_rate_per_types, worker_dur_mean_per_types, worker_variety_per_types = _worker_mobility()
        print(f"MRPAPT: {mob_rate_per_age_per_type}, DMPT: {dur_mean_per_type}, VPT: {variety_per_type}")
        print(f"WMRPT: {worker_mob_rate_per_types}, WDMRT: {worker_dur_mean_per_types}, WVPT: {worker_variety_per_types}")
    else:
        wtype='noness'
        essential_mob_rate_per_type, essential_dur_mean_per_type, essential_variety_per_type = None, None, None
else:
    raise ValueError('Invalid country code.')
    
# convert to average visits per hour per week, to be compatible with simulator
mob_rate_per_age_per_type = np.divide(np.array(mob_rate_per_age_per_type), (24.0 * 7))
worker_mob_rate_per_types = np.divide(np.array(worker_mob_rate_per_types),(24.0*7))

Set `delta`; the setting for delta is explained in the paper.

In [ ]:
# time horizon
delta  = 4.6438 # as set by distributions

In [ ]:
print('Population (by Age): ', population_per_age_group)
print('Sites (by type):     ',  [(np.array(site_type) == i).sum() for i in range(len(mob_rate_per_age_per_type[0]))])

print('Total:', sum(population_per_age_group), len(site_type))

Save arguments for the class object instantiation to be able to initiate `MobilitySimulator` on the fly during inference. That is more efficient than pickling in some cases.

In [ ]:
kwargs = dict(
    home_loc=home_loc, 
    people_age=people_age, 
    site_loc=site_loc, 
    num_people_unscaled=town_population,
    region_population=region_population,
    site_type=site_type, 
    site_dict=site_dict, 
    downsample=downsample,
    mob_rate_per_age_per_type=mob_rate_per_age_per_type,
    daily_tests_unscaled=daily_tests_unscaled, 
    dur_mean_per_type=dur_mean_per_type, 
    variety_per_type=variety_per_type, 
    delta=delta,
    home_tile=home_tile, 
    tile_site_dist=tile_site_dist, 
    people_household=people_household,
    worker_types=worker_types,
    worker_mob_rate_per_types=worker_mob_rate_per_types,
    worker_dur_mean_per_types=worker_dur_mean_per_types,
    worker_work_sites=worker_work_sites,
    social_graph = social_graph,
    num_colleages = num_colleages,
    people_house_site=people_house_site,
    refuse_gathering_rate=refuse_gathering_rate,
    gather_max_size=gather_max_size)

if model_essential==False:
    outfile = f'lib/mobility/{town_name}_settings_{downsample}.pk'
else:
    outfile = f'lib/mobility/{town_name}_settings_{downsample}_type{wtype}_{int(essential_to_total_ratio*100)}pct_social_graph_homesite_new_mob.pk'
with open(outfile, 'wb') as fp:
    pickle.dump(kwargs, fp)
print(f'Saved mobility settings to {outfile}')

Create mobility traces as above, or comment in the last section below to specify fully artifial traces.

In [ ]:
with open("lib/mobility/San_Francisco_settings_100_type0-1-2-3_20pct_social_graph_homesite.pk", 'rb') as fp:
    kwargs = pickle.load(fp)
mob = MobilitySimulator(**kwargs)
mob.verbose = True

In [ ]:
max_time = 17 * 24.0 # e.g. 17 days
%time mob.simulate(max_time=max_time, seed=12345)
# %time mob.to_pickle(f'tu_mobility_{downsample_population}_{downsample_sites}.pk')

In [ ]:
4915 in social_graph.adj[5]

# Laura Experiments

In [ ]:
# def _num_interactions(contact_list):
#     interactions = 0
#     for contact in contact_list:
#         interactions += 

def num_contacts_and_interactions(mob):
    num_contacts = np.array([len(mob.contacts[i]) for i in mob.contacts])
    num_interactions = np.array([sum([len(mob.contacts[i][j]) for j in mob.contacts[i]]) for i in mob.contacts])
    return num_contacts, num_interactions

num_contacts, num_interactions = num_contacts_and_interactions(mob)
print(num_contacts, num_interactions)
print(np.unique(num_interactions))

In [ ]:
len(np.unique(list(mob.contacts[3].keys())))

In [ ]:
len(mob.contacts[3].keys())

In [ ]:
mob.contacts[3]

In [ ]:
import pdb
# len(mob.contacts[3][841])
for person_i in mob.contacts:
    i_contacts = mob.contacts[person_i]
    for person_j in i_contacts:
        inter = i_contacts[person_j]
        if len(inter)>1:
            pdb.set_trace()
            

In [ ]:
def household_size(mob):
    people_household = mob.people_household
    unique_household, counts_household = np.unique(people_household, return_counts=True)
    f = lambda x: counts_household[x]
    household_size = f(people_household)
    return household_size
    # plt.hist(household_size,bins=range(1,9),align='left',rwidth=0.5)
    # plt.xlabel('Household Size')
    # plt.ylabel('Number of Households')

In [ ]:
len(household_size)